In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2018-10-12 11:43:49--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-10-12 11:43:49--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.228, 91.189.89.229
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [5]:
data_1 = pd.read_csv('drive/datasets/jester-jokes/UserRatings1.csv')
data_2 = pd.read_csv('drive/datasets/jester-jokes/UserRatings2.csv')
data_1 = data_1.T
data_2 = data_2.T
data_1 = data_1.dropna()
data_2 = data_2.dropna()
data_2.shape

(1, 100)

In [6]:
data_1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
JokeId,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,...,90.00,91.00,92.00,93.00,94.00,95.00,96.00,97.00,98.00,99.00
User1,5.10,4.90,1.75,-4.17,5.15,1.75,4.76,3.30,-2.57,-1.41,...,5.34,-4.61,3.59,7.18,0.92,6.31,-4.95,-0.19,3.25,4.37
User2,-8.79,-0.87,1.99,-4.61,5.39,-0.78,1.60,1.07,-8.69,-4.66,...,3.59,1.21,2.86,-0.05,-1.75,-1.02,-0.97,4.13,-1.84,2.96
User3,-3.50,-2.91,-2.18,-0.10,7.52,1.26,-5.39,1.50,-8.40,4.37,...,1.84,-4.03,-1.41,1.65,-3.79,3.98,-6.46,-6.89,-2.33,-7.38
User4,7.14,-3.88,-3.06,0.05,6.26,6.65,-7.52,7.28,-5.15,-7.14,...,-4.47,6.36,4.71,-5.19,6.26,3.93,-2.57,1.07,2.33,-0.34


In [7]:
data_2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
JokeId,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0


In [0]:
data_1 = data_1.values

In [9]:
type(data_1)

numpy.ndarray

In [0]:
data = data_1[1:,:]

In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)
print(np.max(data))

1.0000000000000002


In [0]:
inp = tf.placeholder(dtype='float32',shape=[None,100],name='input')
weights = tf.placeholder(dtype='float32',shape=[100,50],name='weights')
forw_bias = tf.placeholder(dtype='float32',shape=[50],name='forward_bias')
back_bias = tf.placeholder(dtype='float32',shape=[100],name='backward_bias')

In [0]:
mat_forw = tf.add(tf.matmul(inp,weights),forw_bias,name='forward_matrix_operation')
mat_forw = tf.nn.sigmoid(mat_forw)
out = tf.nn.relu(tf.sign(mat_forw - tf.random_uniform(tf.shape(mat_forw))))

In [0]:
mat_back = tf.add(tf.matmul(out,tf.transpose(weights)),back_bias,name='backward_matrix_operation')
mat_back = tf.nn.sigmoid(mat_back)
inp_recon_1 = tf.nn.relu(tf.sign(mat_back - tf.random_uniform(tf.shape(mat_back))))
out_recon_1 = tf.nn.sigmoid(tf.add(tf.matmul(inp_recon_1,weights),forw_bias),name='output_reconstruction')
out_recon_1 = tf.nn.relu(tf.sign(out_recon_1 - tf.random_uniform(tf.shape(out_recon_1))))
inp_recon_2 = tf.add(tf.matmul(out_recon_1,tf.transpose(weights)),back_bias)
inp_recon_2 = tf.nn.sigmoid(inp_recon_2)
inp_recon_2 = tf.nn.relu(tf.sign(inp_recon_2 - tf.random_uniform(tf.shape(inp_recon_2))))
out_recon_2 = tf.nn.sigmoid(tf.add(tf.matmul(inp_recon_2,weights),forw_bias),name='output_reconstruction_2')

In [0]:
alpha = 1.0
w_pos_grad = tf.matmul(tf.transpose(inp),out)
w_neg_grad = tf.matmul(tf.transpose(inp_recon_2),out_recon_2)
CD = (w_pos_grad - w_neg_grad)/ tf.to_float(tf.shape(inp)[0])
upadte_w = weights + alpha*CD
up_fo_bia = forw_bias + alpha*tf.reduce_mean(out-out_recon_2,0)
up_ba_bia = back_bias + alpha*tf.reduce_mean(inp-inp_recon_2,0)

In [0]:
err = tf.reduce_mean(tf.square(inp-inp_recon_2))
cur_wts = np.zeros([100,50],dtype='float32')
cur_for_bias = np.zeros([50],dtype='float32')
cur_back_bias = np.zeros([100],dtype='float32')
prv_wts = np.zeros([100,50],dtype='float32')
prv_for_bias = np.zeros([50],dtype='float32')
prv_back_bias = np.zeros([100],dtype='float32')
init = tf.global_variables_initializer()
epochs = 100
batch = 300
errors = []

In [78]:
with tf.Session() as sess:
    for epoch in range(epochs):
        b = 0
        for start,end in zip(range(0,len(data),batch),range(batch,len(data),batch)):
            b = b+1
            xtr = data[start:end]
            cur_wts = sess.run(upadte_w,feed_dict={inp:xtr,weights:prv_wts,forw_bias:prv_for_bias,back_bias:prv_back_bias})
            cur_for_bias = sess.run(up_fo_bia,feed_dict={inp:xtr,weights:prv_wts,forw_bias:prv_for_bias,back_bias:prv_back_bias})
            cur_back_bias = sess.run(up_ba_bia,feed_dict={inp:xtr,weights:prv_wts,forw_bias:prv_for_bias,back_bias:prv_back_bias})
            prv_wts = cur_wts
            prv_for_bias = cur_for_bias
            prv_back_bias = cur_back_bias
            if start%100 == 0:
              
              errors.append(sess.run(err,feed_dict={inp:xtr,weights:prv_wts,forw_bias:prv_for_bias,back_bias:prv_back_bias}))
        print('epoch:',epoch,'error:',errors[-1])
    x = sess.run(inp_recon_1,feed_dict={inp:data[1:2],weights:prv_wts,forw_bias:prv_for_bias,back_bias:prv_back_bias})

epoch: 0 error: 0.30667776
epoch: 1 error: 0.31069884
epoch: 2 error: 0.30757532
epoch: 3 error: 0.3084903
epoch: 4 error: 0.3085399
epoch: 5 error: 0.30942512
epoch: 6 error: 0.30695897
epoch: 7 error: 0.30570832
epoch: 8 error: 0.30162233
epoch: 9 error: 0.29657927
epoch: 10 error: 0.29698938
epoch: 11 error: 0.2955748
epoch: 12 error: 0.29652068
epoch: 13 error: 0.29326686
epoch: 14 error: 0.29500476
epoch: 15 error: 0.29271594
epoch: 16 error: 0.29426154
epoch: 17 error: 0.28710482
epoch: 18 error: 0.28903186
epoch: 19 error: 0.29219407
epoch: 20 error: 0.29187855
epoch: 21 error: 0.2872226
epoch: 22 error: 0.28814268
epoch: 23 error: 0.2791312
epoch: 24 error: 0.28715244
epoch: 25 error: 0.29084128
epoch: 26 error: 0.2886255
epoch: 27 error: 0.28322655
epoch: 28 error: 0.2886597
epoch: 29 error: 0.29080793
epoch: 30 error: 0.29081494
epoch: 31 error: 0.2882561
epoch: 32 error: 0.28650114
epoch: 33 error: 0.2835035
epoch: 34 error: 0.29026684
epoch: 35 error: 0.30004805
epoch: 36 e

In [79]:
g = data[1:2]
prv_wts

array([[-0.27537748, -0.28248104, -0.0306609 , ...,  0.01452793,
        -0.06343006, -0.30133808],
       [-0.2540628 , -0.25790426, -0.10695627, ...,  0.04521807,
        -0.06314653, -0.30504137],
       [-0.11871989, -0.17284067, -0.06413656, ..., -0.00934374,
        -0.00293249, -0.16948171],
       ...,
       [-0.23322201, -0.24245462, -0.19708931, ..., -0.09868377,
        -0.1003563 , -0.31186265],
       [-0.10783426, -0.12391031, -0.19430935, ..., -0.04203366,
        -0.11566246, -0.16759117],
       [-0.23993252, -0.2644088 , -0.15759902, ..., -0.07671626,
        -0.05265778, -0.3501399 ]], dtype=float32)

In [80]:
x-g

array([[ 0.93995859,  0.5300207 , -0.61801242, -0.27639752,  0.20600414,
         0.52536232, -0.59782609, -0.57039337,  0.93478261, -0.27380952,
         0.08540373, -0.90217391, -0.65062112, -0.25362319, -0.57298137,
         0.5326087 , -0.5626294 , -0.33178054,  0.71376812, -0.6257764 ,
         0.15838509,  0.81677019, -0.44720497, -0.27639752,  0.74120083,
         0.27380952,  0.55797101, -0.55020704,  0.22877847,  0.47857512,
         0.17598344, -0.47722567, -0.58022774,  0.80900621,  0.3115942 ,
         0.50258799,  0.26071244, -0.17598344,  0.73602484, -0.58022774,
         0.53519669,  0.60041408,  0.85455487, -0.10817805,  0.62060041,
         0.28881988,  0.64337474,  0.71376812,  0.22101449, -0.70082816,
         0.72360248, -0.57556936, -0.62880743,  0.22619048, -0.22619048,
         0.62060041, -0.00258799, -0.09757357,  0.77380952, -0.14078675,
         0.30383023, -0.71118012,  0.42533471, -0.46221532,  0.6884058 ,
         0.2515528 ,  0.69099379,  0.33178054, -0.7

In [81]:
with tf.Session() as sess:
  bc = sess.run(tf.nn.relu(tf.sign(g - tf.random_uniform(tf.shape(g)))))
  print(bc-x)

[[-1. -1.  1.  0.  0. -1.  0.  1. -1.  0. -1.  1.  0.  1.  0.  0.  1.  0.
   0.  1.  0. -1.  1.  1.  0. -1. -1.  1.  0. -1.  0.  1.  1. -1.  0. -1.
   0.  0. -1.  0.  0. -1. -1.  0.  0.  0. -1. -1.  0.  1. -1.  0.  1.  0.
   0.  0.  0.  0.  0.  0.  0.  1.  0.  1. -1.  0. -1.  0.  0.  0. -1.  0.
   0.  0.  0.  0.  0.  1.  0. -1.  0. -1.  1.  0.  1.  0. -1.  0.  0.  1.
   1.  1.  0.  1.  0.  0. -1.  0. -1. -1.]]


In [82]:
x

array([[1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
        0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
        1., 0., 1., 1.]], dtype=float32)

In [83]:
g

array([[0.06004141, 0.4699793 , 0.61801242, 0.27639752, 0.79399586,
        0.47463768, 0.59782609, 0.57039337, 0.06521739, 0.27380952,
        0.91459627, 0.90217391, 0.65062112, 0.25362319, 0.57298137,
        0.4673913 , 0.5626294 , 0.33178054, 0.28623188, 0.6257764 ,
        0.84161491, 0.18322981, 0.44720497, 0.27639752, 0.25879917,
        0.72619048, 0.44202899, 0.55020704, 0.77122153, 0.52142488,
        0.82401656, 0.47722567, 0.58022774, 0.19099379, 0.6884058 ,
        0.49741201, 0.73928756, 0.17598344, 0.26397516, 0.58022774,
        0.46480331, 0.39958592, 0.14544513, 0.10817805, 0.37939959,
        0.71118012, 0.35662526, 0.28623188, 0.77898551, 0.70082816,
        0.27639752, 0.57556936, 0.62880743, 0.77380952, 0.22619048,
        0.37939959, 0.00258799, 0.09757357, 0.22619048, 0.14078675,
        0.69616977, 0.71118012, 0.57466529, 0.46221532, 0.3115942 ,
        0.7484472 , 0.30900621, 0.66821946, 0.71325052, 0.22619048,
        0.        , 0.52018634, 0.56004141, 0.43